In [148]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
import os
import json

# Census & gmaps API Keys
from config import (weather_key, gkey)
# Configure gmaps
gmaps.configure(api_key=gkey)

In [149]:
#Import Cities from file downloaded from openweather.com
filepath  = "city.list.json"
cities_pd = pd.read_json(filepath, encoding="utf-8")
filepath = "countryCodes.json"
countryCodes = pd.read_json(filepath, orient="index", encoding="utf-8" )
countryCodes.index.rename("country", inplace=True)
countryCodes.columns=["Country Name"]

merged_data = cities_pd.merge(countryCodes, how="inner", on="country")
merged_data.head(10)

,id,name,state,country,coord,Country Name
0,833,Ḩeşār-e Sefīd,,IR,"{'lon': 47.159401, 'lat': 34.330502}","Iran, Islamic Republic Of"
1,3245,Taglag,,IR,"{'lon': 44.98333, 'lat': 38.450001}","Iran, Islamic Republic Of"
2,3530,Qabāghlū,,IR,"{'lon': 46.168499, 'lat': 36.173302}","Iran, Islamic Republic Of"
3,7264,Kalāteh-ye Dowlat,,IR,"{'lon': 57.616982, 'lat': 36.163841}","Iran, Islamic Republic Of"
4,8084,Behjatābād,,IR,"{'lon': 51.461639, 'lat': 36.667431}","Iran, Islamic Republic Of"
5,9874,Ţālesh Maḩalleh,,IR,"{'lon': 50.679192, 'lat': 36.894329}","Iran, Islamic Republic Of"
6,11263,Shahrīār Kandeh,,IR,"{'lon': 53.19902, 'lat': 36.631939}","Iran, Islamic Republic Of"
7,11754,Bālā Aḩmad Kolā,,IR,"{'lon': 52.667271, 'lat': 36.649059}","Iran, Islamic Republic Of"
8,14177,Āqdūz,,IR,"{'lon': 44.26556, 'lat': 39.197498}","Iran, Islamic Republic Of"
9,14256,Āzādshahr,,IR,"{'lon': 48.570728, 'lat': 34.790878}","Iran, Islamic Republic Of"


In [150]:
base_url = "https://api.openweathermap.org/data/2.5/weather?"
params = {"appid": weather_key,
          "units": "imperial"}
#print(params["appid"])
# Sample 10 cities and get their weather info
cities = merged_data.loc[:,["id", "name", "Country Name"]].sample(10)
#print(len(cities))
#print(cities)

weather_data = []

for ind in cities.index:
    print("Fetching weather for city: " , cities["name"][ind] , " in " , cities["Country Name"][ind],"...")
    params["id"] = cities["id"][ind]
    city_weather = requests.get(base_url, params=params).json()
    #print(json.dumps(city_weather, indent=2))
    # print( city_weather["coord"]["lat"],\
    #        city_weather["coord"]["lon"],\
    #        city_weather["main"]["temp"],\
    #        city_weather["main"]["humidity"],\
    #        city_weather["clouds"]["all"],"%",\
    #        city_weather["wind"]["speed"]
    #       )
    weather_data.append({
                     "City ID"    : cities["id"][ind],
                     "Lat"        : city_weather["coord"]["lat"],
                     "Lon"        : city_weather["coord"]["lon"],
                     "Temp"       : city_weather["main"]["temp"],
                     "Humidity"   : city_weather["main"]["humidity"],
                     "Cloudiness" : city_weather["clouds"]["all"],
                     "Wind Speed" : city_weather["wind"]["speed"]
                    })
    
    


Fetching weather for city:  La Baule-Escoublac  in  France ...
Fetching weather for city:  Terme  in  Italy ...
Fetching weather for city:  Thap Than  in  Thailand ...
Fetching weather for city:  Longyearbyen  in  Svalbard And Jan Mayen ...
Fetching weather for city:  Saint-Agnant-de-Versillat  in  France ...
Fetching weather for city:  Great Brickhill  in  United Kingdom ...
Fetching weather for city:  Heathrow  in  United Kingdom ...
Fetching weather for city:  Boscotrecase  in  Italy ...
Fetching weather for city:  Fuentes de Magaña  in  Spain ...
Fetching weather for city:  Atlantida  in  Uruguay ...


In [151]:
#Create a data frame for the data and save to CSV file
weather_df = pd.DataFrame(weather_data)
weather_df = weather_df.set_index("City ID")
weather_df.to_csv("weather_data.csv")
